Imports

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import os
from io import StringIO

Loading the data

In [26]:
file_path = "../data/Solar-Measurements_Malawi_Chileka_WB-ESMAP_Raw_00.dat"
with open(file_path, "r") as file:
    lines = file.readlines()
data_start_index = 4
headers = lines[1].strip().split(",")
data = lines[data_start_index:]
data_io = StringIO("".join(data))
df = pd.read_csv(data_io, names=headers)
df.head()

,"""TIMESTAMP""","""RECORD""","""GHI_Avg""","""DIF_Avg""","""DNI_Avg""","""DNICalc_Avg""","""Temp_Avg""","""RH""","""Rain_Tot""","""WS_Avg""","""WVec_Mag_Avg""","""WD_Avg""","""WD_StdDev""","""WS_Max""","""BP_Avg""","""LoggerTemp_Avg""","""DNI_Calb_Avg""","""Calc_Azimuth""","""Calc_Tilt"""
0,2016-03-16 13:04:00,0,227.0435,223.9821,1.562703,0.061581,27.79,57.43,0.0,5.316,5.159,237.8,13.81,7.448,0,32.11,0,-54.42,23.38
1,2016-03-16 13:05:00,1,228.1676,224.9903,1.267700,3.263819,29.55,57.84,0.0,5.877,5.781,238.7,10.34,7.840,926,32.02,0,-54.42,23.38
2,2016-03-16 13:06:00,2,230.6805,227.2020,1.267695,3.507268,29.58,58.79,0.0,5.856,5.707,239.8,12.90,7.644,926,31.92,0,-54.81,23.58
3,2016-03-16 13:07:00,3,232.6177,228.4676,1.225429,3.751496,29.40,59.77,0.0,6.274,6.195,245.1,9.08,8.040,926,31.80,0,-55.19,23.77
4,2016-03-16 13:08:00,4,235.9392,232.6181,1.267682,4.861165,29.22,60.41,0.0,4.516,4.389,234.6,13.57,5.684,926,31.70,0,-55.57,23.97


Convert timestamp

In [ ]:
# Print column names to verify 'TIMESTAMP' exists
print(df.columns)

print(df.head())

df['"TIMESTAMP"'] = pd.to_datetime(df['"TIMESTAMP"'])

# Set the column as the index
df.set_index('"TIMESTAMP"', inplace=True)

Index(['"TIMESTAMP"', '"RECORD"', '"GHI_Avg"', '"DIF_Avg"', '"DNI_Avg"',
       '"DNICalc_Avg"', '"Temp_Avg"', '"RH"', '"Rain_Tot"', '"WS_Avg"',
       '"WVec_Mag_Avg"', '"WD_Avg"', '"WD_StdDev"', '"WS_Max"', '"BP_Avg"',
       '"LoggerTemp_Avg"', '"DNI_Calb_Avg"', '"Calc_Azimuth"', '"Calc_Tilt"'],
      dtype='object')
           "TIMESTAMP"  "RECORD"  "GHI_Avg"  "DIF_Avg"  "DNI_Avg"  \
0  2016-03-16 13:04:00         0   227.0435   223.9821   1.562703   
1  2016-03-16 13:05:00         1   228.1676   224.9903   1.267700   
2  2016-03-16 13:06:00         2   230.6805   227.2020   1.267695   
3  2016-03-16 13:07:00         3   232.6177   228.4676   1.225429   
4  2016-03-16 13:08:00         4   235.9392   232.6181   1.267682   

   "DNICalc_Avg"  "Temp_Avg"   "RH"  "Rain_Tot"  "WS_Avg"  "WVec_Mag_Avg"  \
0       0.061581       27.79  57.43         0.0     5.316           5.159   
1       3.263819       29.55  57.84         0.0     5.877           5.781   
2       3.507268       29.58  58

Summary statistic and mising value report

In [28]:
summary_stats = df.describe()
missing_values = df.isna().sum()
null_cols_over_5pct = missing_values[missing_values > 0.05 * len(df)]
print("Missing Columns >5%:\n", null_cols_over_5pct)

Missing Columns >5%:
 Series([], dtype: int64)


Outlier detection

In [38]:


cols_to_check = [
    '"GHI_Avg"',
    '"DNI_Avg"',
    '"DNICalc_Avg"',
    '"Temp_Avg"',
    '"WS_Avg"', 
]
z_scores = df[cols_to_check].apply(zscore)
outliers = (z_scores.abs() > 3).any(axis=1)

Drop outlier

In [39]:
df_cleaned = df[~outliers].copy()

# Impute missing values with median
for col in cols_to_check:
    if df_cleaned[col].isna().sum() > 0:
        df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)

Export cleaned data

In [40]:
output_csv = "../data/malawi_clean.csv"
df_cleaned.to_csv(output_csv)

Output directory for plots

In [41]:
output_dir = "data/eda_outputs"
os.makedirs(output_dir, exist_ok=True)

Plots: Time series, Monthly averages, Cleaning impact, Correlation heatmap, Scatter plots, RH scatter plots, Histograms and Bubble chart

In [60]:
# Time series plots
plt.figure(figsize=(12, 6))
df_cleaned[['"GHI_Avg"', '"DNI_Avg"', '"DIF_Avg"', '"Temp_Avg"']].plot()
plt.title("Time Series: GHI, DNI, DHI, Tamb")
plt.savefig(f"{output_dir}/time_series.png")
plt.close()

# Monthly averages
df_cleaned['"Month"'] = df_cleaned.index.month
monthly_avg = df_cleaned.groupby('"Month"')[
    ['"GHI_Avg"','"DNI_Avg"','"DIF_Avg"','"Temp_Avg"']
].mean()
monthly_avg.plot(kind="bar", title="Monthly Averages")
plt.savefig(f"{output_dir}/monthly_averages.png")
plt.close()

# Cleaning impact
# df_cleaned["Cleaning"] = np.random.choice(["Pre", "Post"], size=len(df_cleaned))
# df_cleaned.groupby("Cleaning")[['"ModA"', '"ModB"']].mean().plot(
#     kind="bar", title="Cleaning Impact on ModA & ModB"
# )
# plt.savefig(f"{output_dir}/cleaning_impact.png")
# plt.close()

# Correlation heatmap
corr_cols = ['"GHI_Avg"', '"DNI_Avg"', '"DIF_Avg"','"Temp_Avg"']
sns.heatmap(df_cleaned[corr_cols].corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.savefig(f"{output_dir}/correlation_heatmap.png")
plt.close()

# Scatter plots
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
sns.scatterplot(data=df_cleaned, x='"WS_Avg"', y='"GHI_Avg"', ax=axs[0])
axs[0].set_title("WS vs GHI")
sns.scatterplot(data=df_cleaned, x='"WS_Avg"', y='"GHI_Avg"', ax=axs[1])
axs[1].set_title("WSgust vs GHI")
sns.scatterplot(data=df_cleaned, x='"WD_Avg"', y='"GHI_Avg"', ax=axs[2])
axs[2].set_title("WD vs GHI")
plt.tight_layout()
plt.savefig(f"{output_dir}/scatter_ws_ghi.png")
plt.close()

# RH scatter plots
fig, axs = plt.subplots(1, 2, figsize=(14, 5))
sns.scatterplot(data=df_cleaned, x='"RH"', y='"Temp_Avg"', ax=axs[0])
axs[0].set_title("RH vs Tamb")
sns.scatterplot(data=df_cleaned, x='"RH"', y='"GHI_Avg"', ax=axs[1])
axs[1].set_title("RH vs GHI")
plt.tight_layout()
plt.savefig(f"{output_dir}/scatter_rh.png")
plt.close()

# Histograms
fig, axs = plt.subplots(1, 2, figsize=(14, 5))
df_cleaned['"GHI_Avg"'].hist(ax=axs[0], bins=30)
axs[0].set_title("Histogram of GHI")
df_cleaned['"WS_Avg"'].hist(ax=axs[1], bins=30)
axs[1].set_title("Histogram of WS")
plt.tight_layout()
plt.savefig(f"{output_dir}/histograms.png")
plt.close()

# Bubble chart
plt.figure(figsize=(10, 6))
plt.scatter(
    df_cleaned['"GHI_Avg"'], df_cleaned['"Temp_Avg"'], s=df_cleaned['"RH"'], alpha=0.5
)
plt.xlabel("GHI")
plt.ylabel("Tamb")
plt.title("Bubble Chart: GHI vs Tamb (bubble size = RH)")
plt.savefig(f"{output_dir}/bubble_chart.png")
plt.close()

Step 2